In [1]:
import os; os.chdir('..')
import multiprocessing
from prev_ob_models.Birgiolas2020.fitting import *
import pandas as pd
from multiprocessing import Pool, Process, Queue

/home/justas/anaconda2/envs/p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Could not load NEURONBackend


In [2]:
# MC
mcs = [
    'prev_ob_models.BhallaBower1993.isolated_cells.MC',
    'prev_ob_models.Chen2002.isolated_cells.MC',
    'prev_ob_models.David2008.isolated_cells.MC',
    'prev_ob_models.Davison2000.isolated_cells.MC',
    'prev_ob_models.Djurisic2008.isolated_cells.MC',
    'prev_ob_models.KaplanLansner2014.isolated_cells.MC',
    'prev_ob_models.LiCleland2013.isolated_cells.MC',
    'prev_ob_models.McTavish2012.isolated_cells.MC',
    'prev_ob_models.Migliore2005GJs.isolated_cells.MC',
    'prev_ob_models.Migliore2007columns.isolated_cells.MC',
    'prev_ob_models.Migliore2008microcircuits.isolated_cells.MC',
    'prev_ob_models.Migliore2014bulb3d.isolated_cells.MC',
    'prev_ob_models.Migliore2015operators.isolated_cells.MC',
    'prev_ob_models.MiglioreMcTavish2013.isolated_cells.MC',
    'prev_ob_models.Oconnor2012.isolated_cells.MC1',
    'prev_ob_models.Oconnor2012.isolated_cells.MC2',
    'prev_ob_models.Oconnor2012.isolated_cells.MC3',
    'prev_ob_models.Oconnor2012.isolated_cells.MC4',
    'prev_ob_models.Oconnor2012.isolated_cells.MC5',
    'prev_ob_models.Oconnor2012.isolated_cells.MC6',
    'prev_ob_models.RubinCleland2006.isolated_cells.MC',
    'prev_ob_models.Saghatelyan2005.isolated_cells.MC',
    'prev_ob_models.Shen1999.isolated_cells.MC',
    'prev_ob_models.Short2016.isolated_cells.MC',
    'prev_ob_models.Yu2012.isolated_cells.MC',
    'prev_ob_models.Birgiolas2020.isolated_cells.MC1',
    'prev_ob_models.Birgiolas2020.isolated_cells.MC2',
    'prev_ob_models.Birgiolas2020.isolated_cells.MC3',
    'prev_ob_models.Birgiolas2020.isolated_cells.MC4',
    'prev_ob_models.Birgiolas2020.isolated_cells.MC5',
]

#TC
tcs = [
    'prev_ob_models.Birgiolas2020.isolated_cells.TC1',
    'prev_ob_models.Birgiolas2020.isolated_cells.TC2',
    'prev_ob_models.Birgiolas2020.isolated_cells.TC3',
    'prev_ob_models.Birgiolas2020.isolated_cells.TC4',
    'prev_ob_models.Birgiolas2020.isolated_cells.TC5',
]

#GC
gcs = [
    'prev_ob_models.Davison2003.isolated_cells.GC',
    'prev_ob_models.KaplanLansner2014.isolated_cells.GC',
    'prev_ob_models.LiCleland2013.isolated_cells.GC',
    'prev_ob_models.McTavish2012.isolated_cells.GC',
    'prev_ob_models.Migliore2008microcircuits.isolated_cells.GC',
    'prev_ob_models.Migliore2014bulb3d.isolated_cells.GC',
    'prev_ob_models.Migliore2015operators.isolated_cells.GC',
    'prev_ob_models.MiglioreMcTavish2013.isolated_cells.GC',
    'prev_ob_models.Saghatelyan2005.isolated_cells.GC',
    'prev_ob_models.Short2016.isolated_cells.GC',
    'prev_ob_models.Yu2012.isolated_cells.GC',
    'prev_ob_models.Birgiolas2020.isolated_cells.GC1',
    'prev_ob_models.Birgiolas2020.isolated_cells.GC2',
    'prev_ob_models.Birgiolas2020.isolated_cells.GC3',
    'prev_ob_models.Birgiolas2020.isolated_cells.GC4',
    'prev_ob_models.Birgiolas2020.isolated_cells.GC5'
]

debug = [
    'prev_ob_models.Birgiolas2020.isolated_cells.TC1',
]

In [3]:
def score(q, type, model, clear_fitter_cache):
    fitter = CellFitter(cell_type=type, fitting_model_class=model)
    
    if clear_fitter_cache:
        fitter.clear_cache()
        
    fitter.best = []
    df, score = fitter.get_best_score()
    
    print("%s SCORE: %s"%(fitter.fitting_model_class, score["model_score"]))
    print(df.transpose())
    
    q.put([df, score])

In [4]:
# type: one of 'tc', 'gc', or 'mc'
# array of isolated cell class names e.g. ['prev_ob_models.Chen2002.isolated_cells.MC']
def build_report(type, models, clear_fitter_cache=False):    
    results = []
    
    for model in models:
        q = Queue()
        p = Process(target=score, args=(q, type, model, clear_fitter_cache))
        p.start()
        results.append(q.get())
        p.join()
        
        print('---------------------------------------------------------------')
    
    report = pd.concat([result[0] for result in results]).transpose()

    csv_file = 'all_%s_model_validation_results.csv'%type
    report.to_csv(csv_file)
    print('Saved report to: %s'%os.path.abspath(csv_file))
    
    return report

In [5]:
#build_report('gc', gcs, clear_fitter_cache=False)

In [6]:
build_report('mc', mcs, clear_fitter_cache=False)

('Starting mc FITTER for model:', 'prev_ob_models.BhallaBower1993.isolated_cells.MC')
(<class prev_ob_models.BhallaBower1993.isolated_cells.MC at 0x7f653ac41a78>, 'does not have "params" attribute. No fitting will be performed.')
	1 
<< Connecting sections >>
<< axon connected to soma >>
<< primary dendrite connected to soma >>
<< glomerulus connected to primary dendrite >>
<< glomerulus connected up >>
<< secondary dendrites all connected >>
	1 
	1 
	1 
	0 
	0 
	0 
	0 
	0 
	0 
	0 
	0 
<< Setting membrane properties and inserting channels >>
<< Electrode leak 120 megohms at soma >>
prev_ob_models.BhallaBower1993.isolated_cells.MC SCORE: 7.22505821371
                                     prev_ob_models.BhallaBower1993.isolated_cells.MC
AfterHyperpolarizationAmplitudeTest                            [0.015820688222416803]
AfterHyperpolarizationTimeTest                                       -1.0553476253249
CellCapacitanceTest                                                3.85888651131791

	1 
	1 
	1 
prev_ob_models.Djurisic2008.isolated_cells.MC SCORE: 9.05940922445
                                     prev_ob_models.Djurisic2008.isolated_cells.MC
AfterHyperpolarizationAmplitudeTest                           [1.1455260599876735]
AfterHyperpolarizationTimeTest                                 -1.9873558751133564
CellCapacitanceTest                                            -1.1615321936281167
FISlopeTest                                                    -2.5789473684210527
ISICVTest                                                      -1.1976054688975337
InputResistanceTest                                              3.657210183202147
MembraneTimeConstantTest                                       -0.2513651591537921
ReboundSpikingTest                                                             0.0
RestingVoltageTest                                               0.973959224437883
RheobaseTest                                                     3.803214930363399
SagVolta

model                                prev_ob_models.Migliore2005GJs.isolated_cells.MC
---------------------------------------------------------------
('Starting mc FITTER for model:', 'prev_ob_models.Migliore2007columns.isolated_cells.MC')
(<class 'prev_ob_models.Migliore2007columns.isolated_cells.MC'>, 'does not have "params" attribute. No fitting will be performed.')
prev_ob_models.Migliore2007columns.isolated_cells.MC SCORE: 6.20683939756
                                    prev_ob_models.Migliore2007columns.isolated_cells.MC
AfterHyperpolarizationAmplitudeTest                              [-0.3403774188562392]  
AfterHyperpolarizationTimeTest                                    -0.28502297029311013  
CellCapacitanceTest                                                -0.3920562780345552  
FISlopeTest                                                          1.763157894736842  
ISICVTest                                                          -1.1462393236818258  
InputResistanceTest 

model                                prev_ob_models.Migliore2015operators.isolated_...    
---------------------------------------------------------------
('Starting mc FITTER for model:', 'prev_ob_models.MiglioreMcTavish2013.isolated_cells.MC')
(<class 'prev_ob_models.MiglioreMcTavish2013.isolated_cells.MC'>, 'does not have "params" attribute. No fitting will be performed.')
prev_ob_models.MiglioreMcTavish2013.isolated_cells.MC SCORE: 6.33659165609
                                    prev_ob_models.MiglioreMcTavish2013.isolated_cells.MC
AfterHyperpolarizationAmplitudeTest                              [-0.5394948743226387]   
AfterHyperpolarizationTimeTest                                    -0.09719473427210912   
CellCapacitanceTest                                               -0.43961991084839425   
FISlopeTest                                                          1.894736842105263   
ISICVTest                                                          -1.1453333582478031   
InputR

                                     prev_ob_models.Oconnor2012.isolated_cells.MC4
AfterHyperpolarizationAmplitudeTest                         [-0.39722587095331496]
AfterHyperpolarizationTimeTest                                                 6.0
CellCapacitanceTest                                             -1.073309897327365
FISlopeTest                                                    -2.5789473684210527
ISICVTest                                                      -1.1976054688975337
InputResistanceTest                                            -1.0162168455537999
MembraneTimeConstantTest                                       -1.7032171477490852
ReboundSpikingTest                                                             0.0
RestingVoltageTest                                              0.5548544621188629
RheobaseTest                                                     5.910374898048381
SagVoltageTest                                                   0.686587729348184
Spik

model                                prev_ob_models.Saghatelyan2005.isolated_cells.MC
---------------------------------------------------------------
('Starting mc FITTER for model:', 'prev_ob_models.Shen1999.isolated_cells.MC')
(<class 'prev_ob_models.Shen1999.isolated_cells.MC'>, 'does not have "params" attribute. No fitting will be performed.')
prev_ob_models.Shen1999.isolated_cells.MC SCORE: 4.75083914088
                                     prev_ob_models.Shen1999.isolated_cells.MC
AfterHyperpolarizationAmplitudeTest                       [0.6438566734062907]
AfterHyperpolarizationTimeTest                               1.882700610277651
CellCapacitanceTest                                      -0.005143554960990227
FISlopeTest                                                 0.7105263157894737
ISICVTest                                                 -0.15536109970584994
InputResistanceTest                                        0.24830571999834766
MembraneTimeConstantTest          

model                                prev_ob_models.Birgiolas2020.isolated_cells.MC2
---------------------------------------------------------------
('Starting mc FITTER for model:', 'prev_ob_models.Birgiolas2020.isolated_cells.MC3')
prev_ob_models.Birgiolas2020.isolated_cells.MC3 SCORE: 3.91376156513
                                     prev_ob_models.Birgiolas2020.isolated_cells.MC3
AfterHyperpolarizationAmplitudeTest                            [-1.1117145035665543]
AfterHyperpolarizationTimeTest                                   -0.1030418956026787
CellCapacitanceTest                                               0.5683961248002872
FISlopeTest                                                     -0.34210526315789475
ISICVTest                                                        -0.4914453057162592
InputResistanceTest                                              -0.6066519109056189
MembraneTimeConstantTest                                         -0.4937635722613499
ReboundSpikingTes

,prev_ob_models.BhallaBower1993.isolated_cells.MC,prev_ob_models.Chen2002.isolated_cells.MC,prev_ob_models.David2008.isolated_cells.MC,prev_ob_models.Davison2000.isolated_cells.MC,prev_ob_models.Djurisic2008.isolated_cells.MC,prev_ob_models.KaplanLansner2014.isolated_cells.MC,prev_ob_models.LiCleland2013.isolated_cells.MC,prev_ob_models.McTavish2012.isolated_cells.MC,prev_ob_models.Migliore2005GJs.isolated_cells.MC,prev_ob_models.Migliore2007columns.isolated_cells.MC,...,prev_ob_models.RubinCleland2006.isolated_cells.MC,prev_ob_models.Saghatelyan2005.isolated_cells.MC,prev_ob_models.Shen1999.isolated_cells.MC,prev_ob_models.Short2016.isolated_cells.MC,prev_ob_models.Yu2012.isolated_cells.MC,prev_ob_models.Birgiolas2020.isolated_cells.MC1,prev_ob_models.Birgiolas2020.isolated_cells.MC2,prev_ob_models.Birgiolas2020.isolated_cells.MC3,prev_ob_models.Birgiolas2020.isolated_cells.MC4,prev_ob_models.Birgiolas2020.isolated_cells.MC5
AfterHyperpolarizationAmplitudeTest,[0.015820688222416803],[-0.07959617297745282],[1.12981134305878],[1.238985170069134],[1.1455260599876735],[1.2000402970300663],[3.1811830351974586],[-0.5025807637666787],[0.06962840079017173],[-0.3403774188562392],...,[3.3609277081306477],[1.4121727054895212],[0.6438566734062907],[-0.5025807637666787],[-0.5617686782718747],[-0.11143153754816734],[1.1982335880094601],[-1.1117145035665543],[0.8190349231475493],[0.028450191890919324]
AfterHyperpolarizationTimeTest,-1.0553476253249,-1.2353885351164664,-0.8838480188033329,-0.9703477504488041,-1.9873558751133564,7.308649231070609,3.157489808237756,-0.06164149537150562,-0.16222906514397928,-0.28502297029311013,...,2.138087246346064,-0.010405676585793413,1.882700610277651,-0.06164149537150562,-0.10708043090479395,0.23220624433213993,1.4898281880792374,-0.1030418956026787,-0.3678311813562547,0.375063380165667
CellCapacitanceTest,3.8588865113179143,-0.44073414361031826,5.131308083724331,5.292682888199215,-1.1615321936281167,5.301120228584724,-1.0910373261529565,-0.2901215635751528,-0.9188088468980369,-0.3920562780345552,...,1.3747989919275347,-0.15360973440061365,-0.005143554960990227,-0.2901215635751528,-0.4418740922630638,0.20167712976327928,-0.13673718154646064,0.5683961248002872,0.3341693265099579,-0.09720060176583627
FISlopeTest,-1.7894736842105263,-0.47368421052631576,-2.0526315789473686,-2.1842105263157894,-2.5789473684210527,-1.0,-1.9210526315789471,1.5,2.289473684210526,1.763157894736842,...,-2.5789473684210527,0.18421052631578946,0.7105263157894737,1.5,1.894736842105263,0.3157894736842105,-0.47368421052631576,-0.34210526315789475,-1.9210526315789471,-0.47368421052631576
ISICVTest,0.995530324343946,-0.13298983822104538,0.5814380507819136,0.9565723366238741,-1.1976054688975337,1.9032391324657263,0.4226124223758826,-1.1402865618432387,-1.1456300144195737,-1.1462393236818258,...,0.6809993467536221,-1.1626776146983797,-0.15536109970584994,-1.1402865618432387,-1.1503989017903873,-0.4400278097293899,0.16195262263274512,-0.4914453057162592,-0.5044323899660358,-0.1967247271361703
InputResistanceTest,-0.9584497282803626,0.38090865029181553,-0.09682764655484953,-0.1152542447079869,3.657210183202147,-0.11889364579284462,2.8115584097394843,-0.740662549135059,0.1863404077786491,-0.5056804966462872,...,-2.0014174262774516,0.524738045430607,0.24830571999834766,-0.740662549135059,-0.7220015512065492,0.6134136194521378,0.34523749801534154,-0.6066519109056189,-0.3859521684135235,0.4357821598783553
MembraneTimeConstantTest,0.8502429665061375,-0.601101324079145,3.789180182806229,3.8698104211720783,-0.2513651591537921,3.8698104211720783,-0.37637700075596336,-1.1388054791881421,-1.1655129926400882,-1.058175240822293,...,-1.6501948783959566,-0.19795013224989988,-0.2549292250133838,-1.1388054791881421,-1.237109807338367,0.22287514936372133,-0.27807267260573826,-0.4937635722613499,-0.4126256358854902,-0.19795013224989988
ReboundSpikingTest,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RestingVoltageTest,0.80

In [10]:
build_report('tc', tcs, clear_fitter_cache=False)

('Starting tc FITTER for model:', 'prev_ob_models.Birgiolas2020.isolated_cells.TC1')
prev_ob_models.Birgiolas2020.isolated_cells.TC1 SCORE: 3.82983045548
                                     prev_ob_models.Birgiolas2020.isolated_cells.TC1
AfterHyperpolarizationAmplitudeTest                            [-0.7413212428208619]
AfterHyperpolarizationTimeTest                                   -0.9772565742714951
CellCapacitanceTest                                              -1.6460751191424035
FISlopeTest                                                       0.2361111111111111
ISICVTest                                                        -0.9773314997242611
InputResistanceTest                                               0.4368900983791314
MembraneTimeConstantTest                                         -2.0697674418604652
ReboundSpikingTest                                                               0.0
RestingVoltageTest                                                 1.056989566638

model                                prev_ob_models.Birgiolas2020.isolated_cells.TC5
---------------------------------------------------------------
Saved report to: /home/justas/Repositories/OlfactoryBulb/all_tc_model_validation_results.csv


,prev_ob_models.Birgiolas2020.isolated_cells.TC1,prev_ob_models.Birgiolas2020.isolated_cells.TC2,prev_ob_models.Birgiolas2020.isolated_cells.TC3,prev_ob_models.Birgiolas2020.isolated_cells.TC4,prev_ob_models.Birgiolas2020.isolated_cells.TC5
AfterHyperpolarizationAmplitudeTest,[-0.7413212428208619],[-0.927377487152287],[-0.6199397475980138],[-0.3093240928107219],[0.3991322559925789]
AfterHyperpolarizationTimeTest,-0.9772565742714951,-0.9701492537313404,-0.963515754560528,-0.9594882729211058,-0.9727677402461335
CellCapacitanceTest,-1.6460751191424035,-1.5421781308629694,-0.8704632924214211,-0.829863595835919,-1.6238222999524974
FISlopeTest,0.2361111111111111,-1.4305555555555556,-1.3611111111111112,-2.3333333333333335,-0.5277777777777778
ISICVTest,-0.9773314997242611,-0.6960778171942051,-0.5965807206629256,-0.5844496529107334,-0.7476474369525459
InputResistanceTest,0.4368900983791314,-0.04849168148789316,0.7126686885223598,1.0032981754021404,-0.16929309320303593
MembraneTimeConstantTest,-2.0697674418604652,-1.9534883720930234,-0.6744186046511629,-0.44186046511627924,-2.0697674418604652
ReboundSpikingTest,0.0,-1.0,-1.0,0.0,0.0
RestingVoltageTest,1.056989566638901,-0.0755973169058618,-0.334876178313278,0.46798904294203497,0.9507521707430103
RheobaseTest,0.47095482520753496,-0.24434655728871504,-0.4645508946123669,0.23080774163282483,-0.7635499282992928


In [8]:
#build_report('tc', debug, clear_fitter_cache=True)

In [9]:
# q = Queue()
# score(q, 'tc', debug[0], clear_fitter_cache=True)